In [ ]:
import pandas as pd 
data_train = pd.read_excel('/content/drive/MyDrive/HASOC_2021/HASOC_Dravidian/Task2_Tamil_malayalam_code_mixed/Malayalam_codemixed/Malayalam_offensive_data_Training-YT.xlsx', names=['id','text','label'])
data_val = pd.read_csv('/content/drive/MyDrive/HASOC_2021/HASOC_Dravidian/Task2_Tamil_malayalam_code_mixed/Malayalam_codemixed/malayalam_hasoc_tanglish_test_withlabels.tsv', sep = '\t', names=['id','text','label'])
data_test = pd.read_csv('/content/drive/MyDrive/HASOC_2021/HASOC_Dravidian/Task2_Tamil_malayalam_code_mixed/Malayalam_codemixed/mal-offensive-withoutlabels - Sheet1.tsv', sep = '\t')

In [ ]:
data_train = data_train.dropna()

In [ ]:
data_train.head()

,id,text,label
0,MA_YT001,Thaankal enthaan cheyyarullath?😛,NOT
1,MA_YT002,Ee theetam WCC feminichigalude news aarkk vena...,OFF
2,MA_YT003,fukru nem tiktok oolakale vilich charcha nadat...,OFF
3,MA_YT004,Aashiq abu produce cheytharunnel ee problems u...,NOT
4,MA_YT005,Pennungal oru team aayal ath moonjum ennu epoo...,OFF


In [ ]:
data_val = data_val.dropna()

In [ ]:
data_val.head()

,id,text,label
0,MA_YT5000,Chenkol vendath thanne aayirunnu....,NOT
1,MA_YT5001,Sundardasinte bhakshnam vakkukal ano?,NOT
2,MA_YT5002,Akasha dooth oru copy adi movie anu 'Who will ...,NOT
3,MA_YT5003,Purath onnum pondade... oru pennum payyanum on...,NOT
4,MA_YT5004,Avasanam Fahad oru Oscar medikkumbazhum lalett...,NOT


In [ ]:
data_test.head()

,ID,Tweets
0,MA_YT6001,athe beharyku deputationil pokam pinarai vijay...
1,MA_YT6002,USER cheruparamadathil than thinnunnath alla p...
2,MA_YT6003,USER cheruparamadathilpinaraayi shournuril oru...
3,MA_YT6004,USER vk pinarai thinnunnathu ninte okke theett...
4,MA_YT6005,USER george thante ammade poottile pooda kalay...


In [ ]:
data_train.shape, data_val.shape, data_test.shape

((3999, 3), (951, 3), (999, 2))

In [ ]:
data_train['label'].value_counts()

NOT    2047
OFF    1952
Name: label, dtype: int64

In [ ]:
data_val['label'].value_counts()

OFF    478
NOT    473
Name: label, dtype: int64

In [ ]:
from sklearn import preprocessing
import numpy as np

le = preprocessing.LabelEncoder()

le.fit(data_train['label'])
print(le.classes_)

y_train_index = le.transform(data_train['label'])
y_val_index = le.transform(data_val['label'])

['NOT' 'OFF']


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfvec = TfidfVectorizer(ngram_range=(1,6), analyzer = 'char_wb', max_features = 20000)
#tfidfvec = TfidfVectorizer(ngram_range=(1,3), max_features = 30000)
tfidf_train = tfidfvec.fit_transform(data_train['text'])
tfidf_val = tfidfvec.transform(data_val['text'])

In [ ]:
from keras.utils.np_utils import to_categorical
y_train_cal  =  to_categorical(y_train_index)
y_val_cal  =  to_categorical(y_val_index)


In [ ]:
x_train = tfidf_train
x_test = tfidf_val
y_train = y_train_cal
y_test =y_val_cal

In [ ]:
from keras.models import Sequential 
from keras.layers import Dense, Dropout
model = Sequential()
model.add(Dense(2048, input_dim = tfidf_train.shape[1], activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 2048)              40962048  
_________________________________________________________________
dropout_6 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 512)               1049088   
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 64)                32832     
_________________________________________________________________
dense_15 (Dense)             (None, 2)                 130       
Total params: 42,044,098
Trainable params: 42,044,098
Non-trainable params: 0
__________________________________________

In [ ]:
model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train.toarray(), y_train, validation_data=(x_test.toarray(), y_test), epochs=10, batch_size=20, verbose=2 )

Epoch 1/10
200/200 - 5s - loss: 0.5654 - accuracy: 0.6914 - val_loss: 0.4893 - val_accuracy: 0.7729
Epoch 2/10
200/200 - 3s - loss: 0.2901 - accuracy: 0.8752 - val_loss: 0.5469 - val_accuracy: 0.7434
Epoch 3/10
200/200 - 4s - loss: 0.0904 - accuracy: 0.9660 - val_loss: 0.9003 - val_accuracy: 0.7119
Epoch 4/10
200/200 - 4s - loss: 0.0253 - accuracy: 0.9922 - val_loss: 1.1963 - val_accuracy: 0.7319
Epoch 5/10
200/200 - 4s - loss: 0.0066 - accuracy: 0.9985 - val_loss: 1.4691 - val_accuracy: 0.7361
Epoch 6/10
200/200 - 4s - loss: 0.0021 - accuracy: 0.9997 - val_loss: 1.7189 - val_accuracy: 0.7256
Epoch 7/10
200/200 - 4s - loss: 0.0012 - accuracy: 0.9997 - val_loss: 1.7989 - val_accuracy: 0.7308
Epoch 8/10
200/200 - 3s - loss: 6.2853e-04 - accuracy: 0.9995 - val_loss: 1.8855 - val_accuracy: 0.7256
Epoch 9/10
200/200 - 4s - loss: 4.5333e-04 - accuracy: 0.9995 - val_loss: 1.9743 - val_accuracy: 0.7266
Epoch 10/10
200/200 - 3s - loss: 3.9867e-04 - accuracy: 0.9995 - val_loss: 2.0366 - val_accu

In [ ]:
tfidf_test = tfidfvec.transform(data_test['Tweets'])

In [ ]:
predictions_test_submission = model.predict(tfidf_test.toarray())

In [ ]:
predictions_test_submission

array([[2.5201065e-02, 9.7479898e-01],
       [8.3304799e-01, 1.6695201e-01],
       [1.0000000e+00, 4.1694197e-14],
       ...,
       [2.0875517e-16, 1.0000000e+00],
       [1.0000000e+00, 2.8919619e-08],
       [1.0000000e+00, 7.1520354e-09]], dtype=float32)

In [ ]:
import numpy as np
predictions_test_submission = np.asarray(predictions_test_submission)

In [ ]:
import numpy as np
predictions_test_submission1 = np.zeros_like(predictions_test_submission)
predictions_test_submission1[np.arange(len(predictions_test_submission)), predictions_test_submission.argmax(1)] = 1

In [ ]:
predictions_test_submission1

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [ ]:
predictions_test_submission1_final = np.argmax(predictions_test_submission1, axis=1)

In [ ]:
predictions_test_submission1_final1 = le.inverse_transform(predictions_test_submission1_final)

In [ ]:
data_test['label'] = predictions_test_submission1_final1

In [ ]:
data_test

,ID,Tweets,label
0,MA_YT6001,athe beharyku deputationil pokam pinarai vijay...,OFF
1,MA_YT6002,USER cheruparamadathil than thinnunnath alla p...,NOT
2,MA_YT6003,USER cheruparamadathilpinaraayi shournuril oru...,NOT
3,MA_YT6004,USER vk pinarai thinnunnathu ninte okke theett...,OFF
4,MA_YT6005,USER george thante ammade poottile pooda kalay...,OFF
...,...,...,...
994,MA_YT6995,ente kochu tv dhaivangaleee kaatholnee,OFF
995,MA_YT6996,aaha jhony sins abinayikkuo ith pole,NOT
996,MA_YT6997,ath kaaanich reach vangan nokkunna nee enth na...,OFF
997,MA_YT6998,mattullavarkku vendi kalyanam kazhikkunnavar,NOT


In [ ]:
data_test.to_csv("/content/drive/MyDrive/HASOC_2021/HASOC_Dravidian/Task2_Tamil_malayalam_code_mixed/Malayalam_codemixed/ANU_Task2_Malayalam.tsv", sep="\t", index=False)

In [ ]:
check_submission = pd.read_csv("/content/drive/MyDrive/HASOC_2021/HASOC_Dravidian/Task2_Tamil_malayalam_code_mixed/Malayalam_codemixed/ANU_Task2_Malayalam.tsv", sep="\t")

In [ ]:
check_submission

,ID,Tweets,label
0,MA_YT6001,athe beharyku deputationil pokam pinarai vijay...,OFF
1,MA_YT6002,USER cheruparamadathil than thinnunnath alla p...,NOT
2,MA_YT6003,USER cheruparamadathilpinaraayi shournuril oru...,NOT
3,MA_YT6004,USER vk pinarai thinnunnathu ninte okke theett...,OFF
4,MA_YT6005,USER george thante ammade poottile pooda kalay...,OFF
...,...,...,...
994,MA_YT6995,ente kochu tv dhaivangaleee kaatholnee,OFF
995,MA_YT6996,aaha jhony sins abinayikkuo ith pole,NOT
996,MA_YT6997,ath kaaanich reach vangan nokkunna nee enth na...,OFF
997,MA_YT6998,mattullavarkku vendi kalyanam kazhikkunnavar,NOT
